In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\atusi\appdata\local\temp\pip-req-build-dgoftk5v
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\atusi\AppData\Local\Temp\pip-req-build-dgoftk5v'


In [2]:
import whisper

In [3]:
model = whisper.load_model("small")
result = model.transcribe("C:\dvd.mp3", verbose=True)
print(result["text"])

FileNotFoundError: [WinError 2] 指定されたファイルが見つかりません。

In [4]:
import torch
print(torch.__version__)

2.0.0+cu117


In [5]:
print(torch.cuda.is_available())

True


# いくぞ

In [4]:
!pip install faster-whisper transformers sentencepiece
!python -m pip install -U yt-dlp

     ---------------------------------------- 1.5/1.5 MB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 7.5/7.5 MB 11.7 MB/s eta 0:00:00
     ------------------------------------- 977.6/977.6 kB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 6.7/6.7 MB 9.0 MB/s eta 0:00:00
     -------------------------------------- 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     --------------------------------------- 20.1/20.1 MB 11.1 MB/s eta 0:00:00
  Using cached tokenizers-0.13.3-cp39-cp39-win_amd64.whl (3.5 MB)
     --------------------------------------- 25.3/25.3 MB 11.1 MB/s eta 0:00:00
     -------------------------------------- 266.4/266.4 kB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 11

In [5]:
!yt-dlp -x --audio-format mp3 https://www.youtube.com/live/outcGtbnMuQ?feature=share -o audio.mp3

[youtube] Extracting URL: https://www.youtube.com/live/outcGtbnMuQ?feature=share
[youtube] outcGtbnMuQ: Downloading webpage
[youtube] outcGtbnMuQ: Downloading ios player API JSON
[youtube] outcGtbnMuQ: Downloading android player API JSON
[youtube] outcGtbnMuQ: Downloading m3u8 information
[info] outcGtbnMuQ: Downloading 1 format(s): 251
[download] Destination: audio.webm

[download]   0.0% of   17.93MiB at  190.27KiB/s ETA 01:36
[download]   0.0% of   17.93MiB at  478.64KiB/s ETA 00:38
[download]   0.0% of   17.93MiB at    1.09MiB/s ETA 00:16
[download]   0.1% of   17.93MiB at    2.02MiB/s ETA 00:08
[download]   0.2% of   17.93MiB at    1.61MiB/s ETA 00:11
[download]   0.3% of   17.93MiB at    1.87MiB/s ETA 00:09
[download]   0.7% of   17.93MiB at    2.66MiB/s ETA 00:06
[download]   1.4% of   17.93MiB at    3.75MiB/s ETA 00:04
[download]   2.8% of   17.93MiB at    5.40MiB/s ETA 00:03
[download]   5.6% of   17.93MiB at    6.04MiB/s ETA 00:02
[download]  11.2% of   17.93MiB at    8.64MiB

ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


In [8]:
from faster_whisper import WhisperModel

model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe("audio.webm", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.6fs -> %.6fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'en' with probability 0.998047
[0.000000s -> 12.360000s]  to the GPT-4 developer demo livestream.
[12.360000s -> 13.800000s]  Honestly, it's kind of hard for me
[13.800000s -> 15.400000s]  to believe that this day is here.
[15.400000s -> 18.280000s]  OpenAI has been building this technology really
[18.280000s -> 19.680000s]  since we started the company.
[19.680000s -> 21.080000s]  But for the past two years, we've
[21.080000s -> 24.080000s]  been really focused on delivering GPT-4.
[24.080000s -> 27.420000s]  That started with rebuilding our entire training stack,
[27.420000s -> 29.560000s]  actually training the model, and then


RuntimeError: CUDA failed with error out of memory

In [4]:
import faster_whisper
import math
from tqdm import tqdm

model = faster_whisper.WhisperModel("large-v2", device="cuda")

def convert_to_hms(seconds: float) -> str:
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = math.floor((seconds % 1) * 1000)
    output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
    return output

def convert_seg(segment: faster_whisper.transcribe.Segment) -> str:
    return f"{convert_to_hms(segment.start)} --> {convert_to_hms(segment.end)}\n{segment.text.lstrip()}\n\n"

segments, info = model.transcribe("dvd.mp3")

full_txt = []
timestamps = 0.0  # for progress bar
with tqdm(total=info.duration, unit=" audio seconds") as pbar:
    for i, segment in enumerate(segments, start=1):
        full_txt.append(f"{i}\n{convert_seg(segment)}")
        pbar.update(segment.end - timestamps)
        timestamps = segment.end
    if timestamps < info.duration: # silence at the end of the audio
        pbar.update(info.duration - timestamps)

with open("file.srt", mode="w", encoding="UTF-8") as f:
    f.writelines(full_txt)

Estimating duration from bitrate, this may be inaccurate
100%|██████████| 83.0955/83.0955 [00:43<00:00,  1.91 audio seconds/s]


In [7]:
# インポートとWhisperモデルの設定
from faster_whisper import WhisperModel

model_size = "large-v2"

# WhisperModelインスタンスの作成
model = WhisperModel(model_size, device="cuda", compute_type="float16")
# 音声ファイルをテキストに変換
segments, info = model.transcribe("dvd.mp3", beam_size=5)

text = ''
# 各セグメントからテキストを抽出
for segment in segments:
    text +=segment.text + ('\n')

# 関数: 文末までの文章を結合
def join_sentences_until_period(text):
    sentences = text.strip().split('\n')
    result = []
    i = 0
    while i < len(sentences):
        current_result = []
        while i < len(sentences) and not sentences[i].strip().endswith('.'):
            current_result.append(sentences[i].strip())
            i += 1
        if i < len(sentences):
            current_result.append(sentences[i].strip())
            i += 1
        result.append(' '.join(current_result))
    return result

# 文末までの文章を結合
output = join_sentences_until_period(text)

# 翻訳モデルのインポートとインスタンス化
from transformers import pipeline
fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')

# 各行について翻訳を実行し、結果を出力
for line in output:
    # print(line)
    print(fugu_translator(line)[0]['translation_text'])

KeyboardInterrupt: 

In [6]:
# # 必要なライブラリをインポート
# import faster_whisper
# import math
# from tqdm import tqdm
# from faster_whisper.transcribe import Segment
# from transformers import pipeline

# # Whisperモデルを初期化
# model = faster_whisper.WhisperModel("large-v2", device="cuda")

# # 音声ファイルをテキストに変換する関数
# def transcribe_audio(audio_file):
#     segments, info = model.transcribe(audio_file)
#     return segments, info

# # セグメントをSRT形式の文字列に変換する関数
# def convert_to_srt(segment: Segment, index: int) -> str:
#     return f"{index}\n{convert_to_hms(segment.start)} --> {convert_to_hms(segment.end)}\n{segment.text.lstrip()}\n\n"

# # タイムスタンプをHH:MM:SS,mmm形式に変換する関数
# def convert_to_hms(seconds: float) -> str:
#     hours, remainder = divmod(seconds, 3600)
#     minutes, seconds = divmod(remainder, 60)
#     milliseconds = math.floor((seconds % 1) * 1000)
#     output = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"
#     return output

# # 日本語への翻訳を行う関数
# def translate_to_japanese(text):
#     fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')
#     return fugu_translator(text)[0]['translation_text']

# # 音声ファイルをテキストに変換
# segments, info = transcribe_audio("audio.webm")

# # SRT形式の字幕を作成
# srt_subtitle = []
# timestamps = 0.0

# with tqdm(total=info.duration, unit=" audio seconds") as pbar:
#     for i, segment in enumerate(segments, start=1):
#         srt_subtitle.append(convert_to_srt(segment, i))
#         pbar.update(segment.end - timestamps)
#         timestamps = segment.end
#     if timestamps < info.duration:  # オーディオの最後に無音がある場合
#         pbar.update(info.duration - timestamps)

# # 字幕テキストを結合
# subtitle_text = "\n".join(srt_subtitle)

# # # 字幕を日本語に翻訳
# # translated_subtitle = translate_to_japanese(subtitle_text)
# # テキストを文ごとに分割する関数
# def split_text_into_sentences(text):
#     sentences = text.split('\n')
#     return sentences

# # 日本語への翻訳を行う関数
# def translate_sentences_to_japanese(sentences):
#     fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')
#     translations = [fugu_translator(sentence)[0]['translation_text'] for sentence in sentences]
#     return translations

# # テキストを文ごとに分割
# print("分割中")
# sentences = split_text_into_sentences(subtitle_text)


# # 文を日本語に翻訳
# print("翻訳中")
# translated_sentences = translate_sentences_to_japanese(sentences)

# # 翻訳された文を結合
# print("統合中")
# translated_subtitle = '\n'.join(translated_sentences)


# # 翻訳された字幕をファイルに保存
# print("保存中")
# with open("translated_subtitle.srt", mode="w", encoding="UTF-8") as f:
#     f.write(translated_subtitle)


100%|██████████| 1467.919125/1467.919125 [02:19<00:00, 10.53 audio seconds/s]       


KeyboardInterrupt: 

In [14]:
def seconds_to_srt_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds % 1) * 1000)
    seconds = int(seconds)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{milliseconds:03d}"

def convert_txt_to_srt(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as txt_file:
        lines = txt_file.readlines()

    srt_lines = []
    count = 1

    for line in lines:
        parts = line.strip().split('] ')
        if len(parts) != 2:
            continue

        time_part = parts[0][1:-1]  # Remove '[' and ']'

        # Check if 's' is present and remove it
        if 's' in time_part:
            time_part = time_part.replace('s', '')

        start_time, end_time = time_part.split(' -> ')

        # Convert seconds to SRT time format
        start_time = seconds_to_srt_time(float(start_time))
        end_time = seconds_to_srt_time(float(end_time))

        srt_lines.append(f"{count}\n{start_time} --> {end_time}\n{parts[1]}\n\n")
        count += 1

    with open(output_file, 'w', encoding='utf-8') as srt_file:
        srt_file.writelines(srt_lines)

if __name__ == "__main__":
    input_txt_file = "input.txt"
    output_srt_file = "output5.srt"
    convert_txt_to_srt(input_txt_file, output_srt_file)


# 9/18

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# モデルとトークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# 入力と出力のsrtファイル名を指定
input_srt_file = "input_subtitle.srt"
output_srt_file = "output_subtitle.srt"

# srtファイルを読み取る関数
def read_srt_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    return lines

# srtファイルを書き込む関数
def write_srt_file(file_path, lines):
    with open(file_path, "w", encoding="utf-8") as file:
        file.writelines(lines)

# srtファイルを読み取り、翻訳して結果を新しいsrtファイルに書き込む
def translate_srt_and_write(input_file, output_file):
    srt_lines = read_srt_file(input_file)
    translated_lines = []
    
    for line in srt_lines:
        line = line.strip()
        if line.isdigit():
            # 字幕IDの行はそのまま追加
            translated_lines.append(line)
        elif re.match(r"\d+:\d+:\d+,\d+ --> \d+:\d+:\d+,\d+", line):
            # 時間情報の行もそのまま追加
            translated_lines.append(line)
        elif line:
            # テキスト行を翻訳
            translated_text = translate_to_japanese(line)
            translated_lines.append(translated_text)
        else:
            # 空行はそのまま追加
            translated_lines.append(line)

    # 翻訳結果を新しいsrtファイルに書き込む
    write_srt_file(output_file, translated_lines)

# テキストを翻訳する関数
def translate_to_japanese(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512)
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["jpn_Jpan"], max_length=512
    )
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text

# srtファイルの翻訳と出力
translate_srt_and_write(input_srt_file, output_srt_file)


In [6]:
waypoint = [[1000,0],[1000,1000],[0,1000],[0,0]]
for wayp_xnum in range(len(waypoint)):
    print("目標座標は：x=",waypoint[wayp_xnum][0],", y=",waypoint[wayp_xnum][1], "です")

目標座標は：x= 1000 , y= 0 です
目標座標は：x= 1000 , y= 1000 です
目標座標は：x= 0 , y= 1000 です
目標座標は：x= 0 , y= 0 です


In [5]:
len(waypoint[0])

2